In [7]:
import numpy as np
from scipy import stats

In [1]:
#convert byte to another
import math
def convert_size(size_bytes):
   if size_bytes == 0:
       return "0B"
   size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
   i = int(math.floor(math.log(size_bytes, 1024)))
   p = math.pow(1024, i)
   s = round(size_bytes / p, 2)
   return "%s %s" % (s, size_name[i])

In [5]:
def KL_divergence(p,q):
    return np.sum(
        np.where(np.logical_and(p!=0,q!=0), p* np.log(p/q), 0 )
    )

In [8]:
def Jenson_shanon(p,q):
    p = p / np.linalg.norm(p, ord=1)
    q = q / np.linalg.norm(q, ord=1)
    m = .5 * (p + q)
    return  .5 * ( stats.entropy(p,m) + stats.entropy(q,m) )

In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

def columns_OHC_transforme(DF,columns):
    merged = pd.DataFrame()
    for column_name in columns:
        label_encoder = LabelEncoder()
        encoded = label_encoder.fit_transform(DF[column_name])
        labels =pd.Series(label_encoder.classes_)

        clmt = ColumnTransformer([(column_name,OneHotEncoder(),[0])], remainder='passthrough')
        clmt_fited_data = clmt.fit_transform(DF[[column_name]]) 
        clmt_fited_data = clmt_fited_data if type(clmt_fited_data)== np.ndarray else clmt_fited_data.toarray()
        NDF = pd.DataFrame(clmt_fited_data,columns=[f'{column_name}_{x}' for x in labels]).set_index(DF.index)
        merged = pd.concat([merged,NDF],axis=1)
    DF = DF.drop(columns=columns)
    DF = DF.merge(merged, left_index=True,right_index=True)

    return DF

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
def Kbinser(DF,cols):
    DF = DF.copy()
    for col_name,Nbin in cols:
        est = KBinsDiscretizer(n_bins=Nbin, encode='ordinal', strategy='uniform',random_state=3020)
        DF[col_name] = est.fit_transform( DF[[col_name]] )
    return DF

In [ ]:
from sklearn.preprocessing import LabelEncoder
def label_encoder(DF,cols = []):
    DF = DF.copy()
    for col in cols:
        encoder = LabelEncoder()
        DF[col] = encoder.fit_transform(DF[col])
    return DF